# 인스타그램 로그인
<hr>
아래에 정의된 함수들을 한 번씩만 실시하고, 이 함수를 사용한 코드들을 모아둔 코드에서 크롤링을 실시하기
위의 코드들은 함수를 미리 정의하여, 크롤링 사용에 용이하도록 만든 코드들임

In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import re

# 크롬 브라우저 열기
driver = webdriver.Chrome('C:/Temp/chromedriver')

driver.get("https://www.instagram.com/accounts/login/")
time.sleep(2)

email = '22milky@naver.com'
input_id = driver.find_elements_by_css_selector('input._2hvTZ.pexuQ.zyHYP')[0]
input_id.clear()
input_id.send_keys(email)

password = '09Qhfkadl!' 
input_pw = driver.find_elements_by_css_selector('input._2hvTZ.pexuQ.zyHYP')[1]
input_pw.clear()
input_pw.send_keys(password)
input_pw.submit()

In [2]:
# 인스타그램 해시태그 검색

def insta_searching(word):
    url = "https://www.instagram.com/explore/tags/" + word
    return url
import time

word = "산수갑산"
url = insta_searching(word)
driver.get(url)
time.sleep(5)

In [4]:
# 총 게시물 개수

postnumber = driver.find_elements_by_css_selector("span > span.g47SY")[0].text.replace(',', '')
postnumber = int(postnumber)

In [5]:
# 첫번째 게시물 열기

def select_first(driver):
    first = driver.find_element_by_css_selector("div._9AhH0")
    first.click()
    time.sleep(3)
    
select_first(driver)

In [6]:
# 게시물 내용 크롤링

import re
from bs4 import BeautifulSoup

def get_content(driver):
    
    html = driver.page_source
    soup = BeautifulSoup(html, 'lxml')
    # 본문 내용 
    try:
        content = soup.select('div.C4VMK > span')[0].text
    except:
        content = ' '
    # 해시태그 
    tags = re.findall(r'#[^\s#,\\]+', content)  
    # 작성일자 
    date = soup.select('time._1o9PC.Nzb55')[0]['datetime'][:10]
    # 좋아요 
    try:
        like = soup.select('div.Nm9Fw > button')[0].text[4:-1]   
    except:
        like = 0
    # 위치
    try: 
        place = soup.select('div.M30cS')[0].text
    except:
        place = ''
   
    data = [content, date, like, place, tags]
    return data

get_content(driver)

['"만 오천원짜리 K-샤퀴테리"⠀⠀1.을지로의 간판 순대국은#산수갑산 이지만⠀이미 알만한 사람들은이 집으로 다 갈아탔죠?⠀훨씬 촉촉한 고기와얼큰하고 진하게 끓인 전골🍾⠀⠀2.비주얼 때매모둠순대를 주문했는데⠀그냥 1.5만 짜리 머릿고기만엑기스로 드시는게 이득입니다.⠀K-샤퀴테리,더 맛있게 먹는 꿀팁 하나.⠀따뜻한 기본 국물을앞접시에 좀 덜고 머릿고기를⠀10초 정도만 담가놓으면갓 삶은듯 지방이 살아나요❤⠀⠀3.둘이서 전골까진 부담스럽다면⠀8,000원짜리얼큰탕도 충분히 좋...⠀흠. 솔까 머릿고기+소주에맛가서 국물은 기억이 잘ㅋㅋㅋ⠀⠀⠀⠀#농가순대국 #비터팬을지로#I❤EJ',
 '2021-07-21',
 0,
 '',
 ['#산수갑산', '#농가순대국', '#비터팬을지로', '#I❤EJ']]

In [7]:
# 다음 게시물로 이동

def move_next(driver):

    right = driver.find_element_by_css_selector ('a.coreSpriteRightPaginationArrow')
    right.click()
    time.sleep(3)

move_next(driver)

# 인스타그램 게시물 크롤링 하기    <img src='data/instagram.PNG' width=120 height=120 align="right">         
### : 본격적인 크롤링은 아래의 코드를 사용하기
<hr>
위에 코드들을 한 번 씩 실행 후, 아래의 코드에 word 내용만 변경하여 실행하면 반복 실행 

In [15]:
import pandas as pd

df = pd.read_excel('output/음식점 모음.xlsx').fillna("missing")

for i in range(30) :
    for j in range(20) :
        if df.iloc[i, j] != "missing":
            search = df.iloc[i, j]

            url = insta_searching(search)
            driver.get(url)
            time.sleep(5)

            postnumber = driver.find_elements_by_css_selector("span > span.g47SY")[0].text.replace(',', '')
            postnumber = int(postnumber)

            select_first(driver)
            instagram = []
            data = []

            for i in range(postnumber - 1) :
                get_content(driver)
                instagram = [instagram, data]
                move_next(driver)

,이영자,백종원,성시경,신동엽,최자,비(정지훈),문세윤,김준현,더들리,츄릅켠,수요미식회,생생정보통,전지적참견시점,생활의 달인,6시 내 고향,골목식당,생방송 오늘 저녁,모닝 와이드,맛있는 녀석들,생방송 투데이
0,산골농원(가평),도두반점,하니칼국수,청담동 뜨락,까치둥지,부원냉면,평방옥,낭만짜장,정식당,해운대 시그니엘부산 더뷰,카페 노티드 청담,이성당,성심당 본점,디라이프스타일키친 광화문점,슬지네찐빵 슬지제빵소,레드브릿지,목향원,연경,이성당,레드브릿지
1,엘더버거,삼대천왕 - 지린성,나진국밥,청국장서갈비,뽈살집,안춘선 배추갈비탕,큰맘할매순댓국,서유기짬뽕,신라호텔,서울 약수 금돼지식당,목향원,문개항아리 조천본점,안성휴게소(부산방향),속초붉은대게수산,메밀꽃필무렵,강릉 수제 어묵고로케,이진상회,보스대게,성심당 본점,강릉 수제 어묵고로케
2,홍미닭발,박용채의 대박터진 돈까스,서초동 평양면옥,태화원,오죽이네,노스트레스버거,서운 칼국수,주은감자탕,바이킹스워프,해운대 그랜드조선 부산 뷔페 아리아,애플하우스,장인의집,명동교자 본점,로쏘 1924,원조 유박사 곱창,글린공원,백세삼계탕,마이무,명동교자 본점,글린공원
3,도동집,미미파스타,을지칼국수,갯마을,송파 사랑방,미미미 가든,김북순큰잠비집,원조수구레,워커힐호텔,부산 기장군 바이킹스 워프,다운타우너 한남,야자수마을카페,망향휴게소,배키욘방,새만금횟집,어거스트 힐 강남점,북한산제빵소,진아떡집,씨엘비베이커리,어거스트 힐 강남점
4,아 이맛이야 장작철판구이,삼대천왕 - 고단백식당,시골집(을지로),봉피양샤브,황평집,돝고기,missing,피자헤븐,신라호텔,부산 용호동 차이나반점,서촌스코프,다우랑,금강휴게소(부산방향),다연닭갈비,바로방,삼고집 파주점,베이커리 밤마을,용성통닭 본점,강릉짬뽕순두부 동화가든 본점,삼고집 파주점
5,빵곱창,스트릿푸드파이터 - 짜런쌩 씰롬,춘향미엔,엄마손떡볶이,아노브,소인수서울,missing,태조석갈비,63빌딩,부산 화명동 가쯔렛,엘림들깨수제비칼국수,강릉갈비찜,서울만남의광장(부산방향),르봉뺑,할머니학화호두과자 터미널본점,제주오성갈치조림,수아당,계열사,춘천통나무집닭갈비,제주오성갈치조림
6,육백마지기,거제 중앙식육식당,서대문 양꼬치,태화원,신비갈비살,피셔맨즈,missing,하롱베이의 하루,조선호텔,부산 서면 티파니돈까스,로리스 더 프라임 립,원조호수삼계탕,카페산,솔트24,대선칼국수,칼리오페,엘림들깨수제비칼국수,을밀대 평양냉면,나정순할매쭈꾸미,칼리오페
7,대게 좋은 날,삼대천왕 - 순심원,강남혜장국,토말 본점,일통이반,우미남,missing,배떼기곱창,플라자호텔,부산 광안리 삼오불고기,명동돈가스,푸른회식당,하우스 베이커리,수원의아침,솔밭묵집,카페아스타나,계열사,오성당,빕스 대방점,카페아스타나
8,착한집,엄마네 돼지찌개,청실홍실,70년 소머리 국밥,산수갑산,천지식당,missing,주은감자탕,스테이크,부산 해운대 소문난원조쌈밥,기와집순두부 조안본점,흔들바위,한입소반 숙대본점,불티나만두찐빵,서문우동,러블랑,옹근달,등촌 최월선칼국수,동동국수집,러블랑
9,진대감,간장 닭갈비 공주짐,구반포,청주 육거리 시장,고미테,막내횟집,missing,옛진미칼국수,신라호텔,울산 성남동 쫀드기집,정돈 대학로본점,봉평메밀미가연,쉐이크쉑 청담점,수진분식 성남중앙지하쇼핑몰점,화랑식당,로쏘 1924,가람돌솥밥,강민주의들밥,더아리엘 목동점,로쏘 1924


IndexError: list index out of range

In [ ]:
data_df = pd.DataFrame(data)
data_df.columns = ['content','hashtag','date','like','place']
data_df.to_excel('/resraurant_for_media/output/instagram.csv')